In [1]:
import psycopg2
import pandas as pd

In [2]:
dbname = 'podcast'
username = 'lindsay'

In [3]:
podcastDf = pd.read_pickle('pkl/clean_podcast_data.pkl')

In [4]:
con = psycopg2.connect(database = dbname, user = username)

In [6]:
cursor = con.cursor()

In [7]:
thisPod = podcastDf.iloc[0]
thisPod

artist_id                                                                  NaN
artist_name                                     David Chen and Joanna Robinson
artist_view_url                                                            NaN
artwork_url100               http://is1.mzstatic.com/image/thumb/Music/v4/0...
artwork_url30                http://is1.mzstatic.com/image/thumb/Music/v4/0...
artwork_url60                http://is1.mzstatic.com/image/thumb/Music/v4/0...
artwork_url600               http://is1.mzstatic.com/image/thumb/Music/v4/0...
collection_censored_name           A Cast of Kings - A Game of Thrones Podcast
collection_explicitness                                               explicit
collection_hd_price                                                          0
collection_id                                                        515836681
collection_name                    A Cast of Kings - A Game of Thrones Podcast
collection_price                                    

In [71]:
del podcastDf['id']
del podcastDf['artist_id']
del podcastDf['genre_ids']

In [73]:
# insert podcast data into podcast table
query = "INSERT INTO podcast (artwork_url30, artwork_url60, artwork_url100, artwork_url600, explicit, name, view_url, summary) VALUES (%s, %s, %s, %s, %s, %s, %s, %s) RETURNING id;"
podcast_id = []
for ind, thisPod in podcastDf.iterrows():
    data = (thisPod['artwork_url30'], thisPod['artwork_url60'], thisPod['artwork_url100'], thisPod['artwork_url600'], thisPod['content_advisory_rating'], thisPod['collection_censored_name'], thisPod['collection_view_url'], thisPod['podcast_summary'])
    cursor.execute(query, data)
    podcast_id.append(cursor.fetchone()[0])
    
con.commit()
podcastDf['podcast_id'] = podcast_id

In [18]:
# get unique artists
artistDf = podcastDf[['artist_name', 'artist_view_url']]
artistDf = artistDf.drop_duplicates()
artistDf.shape

(4863, 2)

In [74]:
# insert artist data into artist table
query = "INSERT INTO artist (view_url, name) VALUES (%s, %s) RETURNING id;"
artist_id = []
for ind, row in artistDf.iterrows():
    data = (row['artist_view_url'], row['artist_name'])
    cursor.execute(query, data)
    artist_id.append(cursor.fetchone()[0])
con.commit()
artistDf['artist_id'] = artist_id

In [48]:
# get unique genres
genreDf = podcastDf['genres']
genreList = []
for row in genreDf:
    for genre in row:
        genreList.append(genre)
genreList = list(set(genreList))

In [76]:
# insert genre data into genre table
query = "INSERT INTO genre (name) VALUES (%s) RETURNING id;"
genre_id = []
for item in genreList:
    data = (item, )
    cursor.execute(query, data)
    genre_id.append(cursor.fetchone()[0])
con.commit()
genreDf = pd.DataFrame({'name' : genreList,
                       'genre_id': genre_id})

In [77]:
genreDf

,genre_id,name
0,137,Arts
1,138,Philosophy
2,139,Tech News
3,140,Personal Journals
4,141,Podcasting
5,142,Games & Hobbies
6,143,Science & Medicine
7,144,Performing Arts
8,145,Sports & Recreation
9,146,TV & Film
